In [1]:
import xlwings as xw

In [11]:
# Input file paths
lokesh_file = "./data/lokesh/Fall2019_ASTR116-10_Essay1_anonymized_Aspirational_tacited.xlsx"
reina_file = "./data/reina/Fall2019_ASTR116-10_Essay1_anonymized_Aspirational_tacited_reina.xlsx"

# Load both workbooks
lokesh_wb = xw.Book(lokesh_file)
reina_wb = xw.Book(reina_file)

# Get the sheets from both workbooks
lokesh_sheet = lokesh_wb.sheets.active
reina_sheet = reina_wb.sheets.active

# Create a new workbook and sheet for the combined data
combined_wb = xw.Book()
combined_sheet = combined_wb.sheets.active


If their annotations are matched,

- don’t format any annotations.
- Column M (theme present) should be populated with boolean Yes or No (Yes=Theme Present, No=No Theme Present)
  - Yes or No
- Column P (final annotated essay)
  - If Theme Present = Yes
    - Merge the red annotations of both Lokesh and Reina’s using Visual Basic.
    - Return only the red annotations (similar to when we humanly copy-paste annotations that are relevant).
  - If Theme Present = No
    - return 0

If their annotations are mismatched,

- highlight their annotations in **red** to signify mismatch and write "X"
- Gian has to reconcile the mismatch manually
  - With the output, Gian will resolve the mismatch by going the spreadsheet(s).


In [4]:
# (Re)-naming columns
reina_sheet.range("K1").value = "Reina Aspirational Present"
reina_sheet.range("L1").value = "Reina Annotated Essay"
reina_sheet.range("M1").value = "Reina Comments"

lokesh_sheet.range("K1").value = "Lokesh Aspirational Present"
lokesh_sheet.range("L1").value = "Lokesh Annotated Essay"
lokesh_sheet.range("M1").value = "Lokesh Comments"

combined_sheet.range("K1").value = "Reina Label"
combined_sheet.range("L1").value = "Lokesh Label"
combined_sheet.range("M1").value = "Reconciled"
combined_sheet.range("P1").value = "Reina Red-only Annotation"
combined_sheet.range("Q1").value = "Lokesh Red-only Annotation"
combined_sheet.range("R1").value = "Reconciled Red-only Annotation"


### Reconciliing Annotations: 4 Cases

1. Case that Lokesh and Reina disagrees
    * One of them has a 0 ("No")
2. Case that Lokesh and Reina agrees (negative theme)
    * They have the same annotation, which is 0.
3. Case that Lokesh annd Reina agrees (positive theme) AND have similar highlighted text
    * They have the same annotation and the same red-only annotation.
4. Case that Lokesh and Reina agrees (postiive theme) BUT have different highlighted text
    * They have the same "Yes" labels, but have different highlighted text -> merge_stings

In [12]:
def process_reconciled_labels(k_col, l_col, m_col):
    """
    Reconciling labels
    input: 
    * reina's labels -> k_col
    * lokesh's lables -> l_col
    * reconciled column range -> m_col
    """
    for k_cell, l_cell, m_cell in zip(k_col[1:], l_col[1:], m_col):
        # if reina's labels match lokesh's labels
        if k_cell.value == l_cell.value:
            # the reconciled value is the same matching value
            m_cell.value = k_cell.value
        # if not
        else:
            # then the reconciled value is undetermined 
            m_cell.value = "X"
            m_cell.font.color = (0,0,0)
            m_cell.color = (255, 0, 0)  # Set font color to red


def merge_strings(a, b):
    """
    Helper function to merge two similar strings
    To be used for Case 4
    """
    # Split strings into lists of words
    words_a = a.split()
    words_b = b.split()

    # Find the common prefix
    common_prefix = []
    for word_a, word_b in zip(words_a, words_b):
        if word_a == word_b:
            common_prefix.append(word_a)
        else:
            break

    # Join the common prefix and the remaining words from b
    merged_string = " ".join(common_prefix + words_b[len(common_prefix):])

    return merged_string


# merged_string = merge_strings(cell_n15, cell_o15)

# #Return the string to P
# print(merged_string)

def process_reconciled_annotations(n_col, o_col, p_col, q_col, r_col):
    """
    Reconciling annotations
    input: 
    * reina's annotations -> n_col
    * lokesh's annotations -> o_col
    * reina's red-only annotations -> p_col
    * lokesh's red-only annotations -> q_col
    * reconciled column r range -> r_col
    """

    # For each value in n_col, o_col, p_col, q_col, r_col
    for n_cell, o_cell, p_cell, q_cell, r_cell in zip(n_col[1:], o_col[1:], p_col, q_col, r_col):

        # Get the cell addresses of Reina and Lokesh's annotations 
        n_cell_address = n_cell.address # The address of Reina's annotation cell ex. N1
        o_cell_address = o_cell.address # Address of Lokesh's annotation cell ex. O1
        p_cell_address = p_cell.address # Address of cell at column P, this is where we will put only the red annotations of Reina
        q_cell_address = q_cell.address # Address of cell at column Q, this is where we will put only the red annotations of Lokesh
        r_cell_address = r_cell.address # Address of cell at column R, this is where we will put the reconciled red text from Reina and Lokesh

        # Pass these cell addresses to get_color_text(x)
        n_cell_color_only = f"=IF(GetColorText({n_cell_address})=\"\",0,GetColorText({n_cell_address}))"
        o_cell_color_only = f"=IF(GetColorText({o_cell_address})=\"\",0,GetColorText({o_cell_address}))"

        combined_sheet.range(p_cell_address).value = n_cell_color_only
        combined_sheet.range(q_cell_address).value = o_cell_color_only

        # print(n_cell_color_only)

        p_cell_value = combined_sheet.range(p_cell_address).value
        q_cell_value = combined_sheet.range(q_cell_address).value
        r_cell_value = combined_sheet.range(r_cell_address).value

        print(p_cell_value)
        print(q_cell_value)
        print(r_cell_value)
      
        # Case 2 and 3
        if (p_cell_value == q_cell_value): # Return 0 or that red-only text
            r_cell.value = p_cell_value
        # Case 4:
        elif (p_cell_value != q_cell_value) and p_cell_value != 0 and q_cell_value != 0:   
            r_cell.value = merge_strings(p_cell_value, q_cell_value)
        # Case 1: 
        else:
            # r_cell value will n_cell or o_cell depending if whether it's a string or not
            r_cell.value = n_cell.value if isinstance(p_cell.value, str) else o_cell.value if isinstance(q_cell.value, str) else 0
            r_cell.font.color = (0,0,0)
            r_cell.color = (255,255,0)  # Set cell color to yellow

In [13]:
def reconcile(book_1_path, book_2_path):
    # Load both workbooks
    wb_1 = xw.Book(book_1_path) # Reina
    wb_2 = xw.Book(book_2_path) # Lokesh

    # Get the sheets from both workbooks
    sheet_1 = wb_1.sheets.active
    sheet_2 = wb_2.sheets.active

    # Combined sheet
    wb_3 = xw.Book()
    sheet_3 = wb_3.sheets.active

    # Renaming columns
    # Reina's sheets
    sheet_1.range("K1").value = "Reina Aspirational Present"
    sheet_1.range("L1").value = "Reina Annotated Essay"
    sheet_1.range("M1").value = "Reina Comments"

    # Lokesh's sheets
    sheet_2.range("K1").value = "Lokesh Aspirational Present"
    sheet_2.range("L1").value = "Lokesh Annotated Essay"
    sheet_2.range("M1").value = "Lokesh Comments"

    # Combined sheet
    sheet_3.range("K1").value = "Reina Label"
    sheet_3.range("L1").value = "Lokesh Label"
    sheet_3.range("M1").value = "Reconciled"
    sheet_3.range("P1").value = "Reina Red-only Annotation"
    sheet_3.range("Q1").value = "Lokesh Red-only Annotation"
    sheet_3.range("R1").value = "Reconciled Red-only Annotation"

    # Populating combined sheet
    # Retrieving labels ("Yes" or "No") from Reina and Lokesh's sheets
    reina_labels = sheet_1.range("K1").expand("down")
    lokesh_labels = sheet_2.range("K1").expand("down")
    reina_annotations = sheet_1.range("L1").expand("down") 
    lokesh_annotations = sheet_2.range("L1").expand("down")

    # Copy over information from Reina and Lokesh's workbooks to the combined sheet
    sheet_1.range('A1:K1').expand("down").copy(sheet_3.range('A1:K1'))
    sheet_2.range('K1').expand("down").copy(sheet_3.range("L1"))
    reina_annotations.copy(sheet_3.range("N1"))
    lokesh_annotations.copy(sheet_3.range("O1"))

    # Initiate the column variables we will be working with in combined sheet
    reina_labels = sheet_3.range("K1").expand("down")
    lokesh_labels = sheet_3.range("L1").expand("down")
    reconciled_labels = sheet_3.range("M:M")[1:] # M column, this is where we will put all of the reconciled labels
    
    reina_annotations = sheet_3.range("N1").expand("down") # N column
    lokesh_annotations = sheet_3.range("O1").expand("down") # O column
    column_p_range = sheet_3.range("P:P")[1:] # P column, Reina's red-only text
    column_q_range = sheet_3.range("Q:Q")[1:] # Q column, Lokesh's red-only text
    column_r_range = sheet_3.range("R:R")[1:] # R column, Reconciled red text

    # Run functions to process labels and annotations
    process_reconciled_labels(reina_labels, lokesh_labels, reconciled_labels)
    process_reconciled_annotations(reina_annotations, lokesh_annotations, column_p_range, column_q_range, column_r_range)
    
    sheet_3.save("./data/reconciliation_temp/sheet3.xlsm")
    sheet_3.close()
    



## Reconciliation Driver

In [ ]:
"""
Comparing Reina and Lokesh's labels
"""

reina_labels = reina_sheet.range("K1").expand("down")

reina_sheet.range('A1:K1').expand("down").copy(combined_sheet.range('A1:K1'))

lokesh_labels = lokesh_sheet.range("K1").expand("down")

lokesh_sheet.range('K1').expand("down").copy(combined_sheet.range("L1"))


combined_sheet.range("M1").value = "Reconciled"


column_c_range = combined_sheet.range("M:M")[1:]

# For each value in reina's labels and lokesh's labels
for a_cell, b_cell, c_cell in zip(reina_labels[1:], lokesh_labels[1:], column_c_range):
    # if reina's labels match lokesh's labels
    if a_cell.value == b_cell.value:
        # the reconciled value is the same matching value
        c_cell.value = a_cell.value
    # if not
    else:
        # then the reconciled value is undetermined 
        c_cell.value = "X"
        c_cell.font.color = (0,0,0)
        c_cell.color = (255, 0, 0)  # Set font color to red

# Range object of reina's annotations 
reina_annotations = reina_sheet.range("L1").expand("down")
# To see as a list
# reina_annotation.value

# Range object of lokesh's annotations
lokesh_annotations = lokesh_sheet.range("L1").expand("down")

# Copy their annotations from their respective files to the combined sheet.
# In the combined sheet, their reina and lokesh's annotations are stored in columns N and O, respectively
reina_annotations.copy(combined_sheet.range("N1"))
lokesh_annotations.copy(combined_sheet.range("O1"))

column_p_range = combined_sheet.range("P:P")[1:]


# column M values
reconciled_labels = combined_sheet.range("M1").expand("down")
reina_annotations = combined_sheet.range("N1").expand("down") # N column
lokesh_annotations = combined_sheet.range("O1").expand("down") # O column
column_p_range = combined_sheet.range("P:P")[1:] # P column, Reina's red-only text
column_q_range = combined_sheet.range("Q:Q")[1:] # Q column, Lokesh's red-only text
column_r_range = combined_sheet.range("R:R")[1:] # R column, Reconciled red text


In [77]:
# UPDATE 
# Column P is now the temporary red only lokesh text
# Column Q is now the temporary red only reina text

In [6]:
# Check if the VBA function GetColorText is working

# def get_color_text(x):
#     return f"=GetColorText({x})"



# combined_sheet.range("P4").value = get_color_text("$N$4")
# combined_sheet.range("P4").value


'I am here because I want to expand my understanding of the world so that I am capable of impacting it in some way By taking classes and ultimately earning a degree, I am empowering myself to be able to think critically about the issues we face and find ways to change that reality Hopefully, this will lead me to a career in international politics that is as fulfilling as it is impactful I do not know what exactly that career will look like or what I will become the most passionate about, but for the time being, I am here to learn Educations is a powerful tool that is not available to everyone and I am here to take advantage of that opportunity and use it to reach my full potential as an individual In this sense, I am here to become a better, more empathetic, more knowledgeable person'

In [35]:
combined_wb.save("./data/reconciliation_temp/combined.xlsm")
combined_wb.close()

## There's 4 cases to look out for when returning the text for column P (merged annotated text)

1. Case that Lokesh and Reina disagrees
2. Case that Lokesh and Reina agrees (negative theme)
3. Case that Lokesh annd Reina agrees (positive theme) AND have similar highlighted text
4. Case that Lokesh and Reina agrees (postiive theme) BUT have different highlighted text

In [8]:
reina_annotations = combined_sheet.range("N1").expand("down")
lokesh_annotations = combined_sheet.range("O1").expand("down")
column_p_range = combined_sheet.range("P:P")[1:] # Reina's red-only text
column_q_range = combined_sheet.range("Q:Q")[1:] # Lokesh's red-only text
column_r_range = combined_sheet.range("R:R")[1:] # Reconciled red text

## There's 4 cases to look out for when returning the text for column P (merged annotated text)

1. Case that Lokesh and Reina disagrees == return yellow cell
2. Case that Lokesh and Reina agrees (negative theme) == return 0
3. Case that Lokesh annd Reina agrees (positive theme) AND have similar highlighted text == return that text
4. Case that Lokesh and Reina agrees (postiive theme) BUT have different highlighted text == return merge of that text 

In [10]:
process_reconciled_annotations(reina_annotations, lokesh_annotations, column_p_range, column_q_range, column_r_range)

combined_wb.save("./data/reconciliation_temp/combined.xlsm")
combined_wb.close()

0.0
 I plan on working in the world of cinema since I fell in love with the world of theater and I can create, imagine, and do whatever I want where I’m not bound to the laws of physics
None
0.0
0.0
None
I am here because I want to expand my understanding of the world so that I am capable of impacting it in some way By taking classes and ultimately earning a degree, I am empowering myself to be able to think critically about the issues we face and find ways to change that reality Hopefully, this will lead me to a career in international politics that is as fulfilling as it is impactful I do not know what exactly that career will look like or what I will become the most passionate about, but for the time being, I am here to learn Educations is a powerful tool that is not available to everyone and I am here to take advantage of that opportunity and use it to reach my full potential as an individual In this sense, I am here to become a better, more empathetic, more knowledgeable person
I 

In [44]:


cell_n15 = combined_sheet["N14"]
cell_o15 = combined_sheet["O14"]

cell_n15 = get_color_text(cell_n15)
cell_o15 = get_color_text(cell_o15) 

print(cell_n15 == cell_o15) #False

# Then merge the strings
def merge_strings(a, b):
    # Split strings into lists of words
    words_a = a.split()
    words_b = b.split()

    # Find the common prefix
    common_prefix = []
    for word_a, word_b in zip(words_a, words_b):
        if word_a == word_b:
            common_prefix.append(word_a)
        else:
            break

    # Join the common prefix and the remaining words from b
    merged_string = " ".join(common_prefix + words_b[len(common_prefix):])

    return merged_string


merged_string = merge_strings(cell_n15, cell_o15)

#Return the string to P
print(merged_string)



False
=GetColorText(<Range [Book1]Sheet1!$O$14>)


In [189]:
a = get_color_text(cell_n5)
a

' This class will help me better understand the universe I am here to gain perspective I desire to begin to grasp the the balance of matter\xa0 \xa0 \xa0I am here to learn and to explore I seek balance, shadow and light, ebb and flow, waning and waxing, within my daily universe'

In [190]:
b= get_color_text(cell_o5) 
b

' This class will help me better understand the universe I feel hopeful in my tiny place.  I am here to gain perspective I desire to begin to grasp the the balance of matter\xa0 \xa0 \xa0I am here to learn and to explore I seek balance, shadow and light, ebb and flow, waning and waxing, within my daily universe'

This class will help me better understand the universe I feel hopeful in my tiny place. I am here to gain perspective I desire to begin to grasp the the balance of matter I am here to learn and to explore I seek balance, shadow and light, ebb and flow, waning and waxing, within my daily universe


In [124]:
reina_sheet["L4"].characters.text

k.missing_value

In [89]:
combined_sheet["B1"]

<Range [combined.xlsx]Sheet1!$B$1>

In [92]:
combined_sheet["N2"].characters.text

k.missing_value

In [141]:
get_color_text = combined_wb.macro("GetColorText")
get_color_text(14,4)

CommandError: Command failed:
		OSERROR: -1728
		MESSAGE: The object you are trying to access does not exist
		COMMAND: app(pid=79082).workbooks['combined.xlsx'].name.get()

In [144]:
combined_wb = xw.Book("./data/reconciliation_temp/combined.xlsm")
combined_sheet = combined_wb.sheets.active



FileNotFoundError: No such file: './data/reconciliation_temp/combined.xlsm'